In [ ]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
nltk.download('punkt_tab')
nltk.download('stopwords')

In [ ]:
complaints_df = pd.read_csv('complaints.csv')

In [ ]:
complaints_df['Date received'] = pd.to_datetime(complaints_df['Date received'], format='%m/%d/%Y', errors='coerce')
complaints_df['Date sent to company'] = pd.to_datetime(complaints_df['Date sent to company'], format='%m/%d/%Y', errors='coerce')

In [ ]:
if 'Tags' in complaints_df.columns:
    complaints_df['Tags'] = complaints_df['Tags'].str.split(', ')
    complaints_df = complaints_df.explode('Tags')

In [ ]:
complaints_df = complaints_df.replace('', np.nan)

In [ ]:
for col in complaints_df.select_dtypes(include='object').columns:
    complaints_df[col] = complaints_df[col].replace('N/A', np.nan)

In [ ]:
unique_counts = complaints_df.nunique()
print("\nUnique value counts per column:")
print(unique_counts)

In [ ]:
top_5_companies = complaints_df['Company'].value_counts().head(5)
print("\nTop 5 companies with the most records:")
print(top_5_companies)

In [ ]:
if 'Consumer complaint narrative' not in complaints_df.columns:
    print("\nError: 'Consumer complaint narrative' column not found. Cannot proceed with sentiment analysis.")
    exit()

In [ ]:
jpm_complaints_df = complaints_df[complaints_df['Company'] == "JPMORGAN CHASE & CO."].copy()
jpm_complaints_df['tokenized_narrative'] = jpm_complaints_df['Consumer complaint narrative'].astype(str).apply(word_tokenize)

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
jpm_complaints_df['sentiment_scores'] = jpm_complaints_df['Consumer complaint narrative'].astype(str).apply(sid.polarity_scores)
jpm_complaints_df['compound_sentiment'] = jpm_complaints_df['sentiment_scores'].apply(lambda x: x['compound'])
jpm_complaints_df['negative_sentiment'] = jpm_complaints_df['sentiment_scores'].apply(lambda x: x['neg'])
stop_words = set(stopwords.words('english'))
negative_words_list = []
for index, row in jpm_complaints_df.iterrows():
    if row['negative_sentiment'] > 0.5: # A threshold for 'highly negative'
        tokens = [word.lower() for word in row['tokenized_narrative'] if word.isalpha() and word.lower() not in stop_words]
        negative_words_list.extend(tokens)
negative_word_count = Counter(negative_words_list) # Top 50 negative words
print("\nTop negative words for JPMorgan Chase & Co. (based on VADER):")
print(negative_word_count)

In [ ]:
if negative_word_count:
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(negative_word_count))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Negative Word Cloud for JPMorgan Chase & Co. Complaints')
    plt.show()
else:
    print("No sufficiently negative words found to generate a word cloud.")

In [ ]:
product_sentiment = jpm_complaints_df.groupby('Product')['compound_sentiment'].mean().reset_index()
product_sentiment = product_sentiment.sort_values(by='compound_sentiment', ascending=False)

plt.figure(figsize=(12, 7))
sns.barplot(x='Product', y='compound_sentiment', data=product_sentiment, palette='viridis')
plt.title('Average Compound Sentiment by Product for JPMorgan Chase & Co.')
plt.xlabel('Product')
plt.ylabel('Average Compound Sentiment')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()